# Optimization of 2-well QCL by sequential IV - Gain

This scirpt was used for optimizing 2-well THz QCLs with the following parameters:

Bias range from 50-70 meV

Frequency range from 10-18 meV

Including e-e scattering

Interpolation of IV and gain

Gain evaluated at maximum of IV curve

Varying x synchronously for all barriers

Using multi-dimensional Gaussian Process optimizer

Varying x around 28.5%

Varying layer widths by 20% from structure #366 from previous optimization of EV2624 (Bosco et al., APL 2019: https://aip.scitation.org/doi/10.1063/1.5110305)

In [ ]:
from aftershoq.interface import Inegf
from aftershoq.structure import Sgenerator
from aftershoq.qcls import Franckie2well366_2019
from aftershoq.numerics.runplatf import Euler
from aftershoq.numerics import MDGaussopt
from aftershoq.structure.sgenerator import comp_modes
import aftershoq.utils.systemutil as su
import numpy as np
from scipy.interpolate import interp1d
from matplotlib import pyplot as pl
import os
import aftershoq.utils.debug as dbg

In [ ]:
dbg.verbosity = dbg.verb_modes["silent"]

#model = Inegf(pltfm=Euler(24,"4:00", paral_in=Euler.paral_modes["OMP"]))
model = Inegf(pltfm=Euler(48,"4:00"))

s = Franckie2well366_2019(T=300)

path="/cluster/home/framarti/work/2well_opt/seq-MGgauss_try1-366/"
su.mkdir(path)

model.numpar["Nper"],model.numpar["Nstates"] = 2,7
model.numpar["NE"],model.numpar["Nk"] = 800, 600
model.numpar["Tlattice"] = 300
model.numpar["efield0"], model.numpar["defield"], model.numpar["Nefield"] = 0.050, 0.005, 5
model.numpar["Nhist"], model.numpar["maxits"] = 40,40
model.numpar["use-e-e"] = True
model.numpar["gen"] = 5e-4

npg = model.numpar.copy()

npg["Nefield"] = 1
npg["Nh"] = 1
npg["efac0"] = 1e-5
npg["omega0"], npg["domega"], npg["Nomega"] = 0.010, 0.002, 6
npg["boolEins"] = True

print(model.numpar)

# vary layers 20%
dw = [l.width*0.20 for l in s.layers]
dw[4] = 0

print(dw)
dx = []
for l in s.layers:
    x = l.material.x
    if x is None:
        x = 0
        dx.append(0)
    else:
        dx.append(0.015)
        l.material.updateAlloy(x=0.285)

In [ ]:
sg = Sgenerator(origstruct=s, dx=dx, dw=dw, comp_mode=comp_modes["LINKED ALL"])
try:
    old_s = model.loadAllStructures(path, s)
    sg.structures = old_s
except FileNotFoundError:
    coords = sg.genRanStructs(N=20)

In [ ]:
t = model.runStructSeq(structures=sg.structures, path=path,seq=[model.numpar, npg], runwannier=True)
for tt in t: tt.join()
model.gatherResults(structures=sg.structures, pathwd=path, runhdiag=False, runbandplot=False)

In [ ]:
pathwd = path

limits = sg.getLimits()

opt = MDGaussopt(tolerance = 0.0, procmax = 10, maxiter = 150, limits = limits)

x0, y0 = opt.addEvaldPoints(model, sg, pathwd, sg.structures)

conv = opt.minimize(model, sg, pathwd, seq=[model.numpar, npg])